# Challenge Four

In [1]:
import os
import logging
import vertexai
from typing import List, Dict, Any, Optional
from pathlib import Path
from dotenv import load_dotenv

env_path = Path.cwd().parent / '.env'
load_dotenv(env_path)

logger = logging.getLogger(__name__)
logging.basicConfig(filename="app.log", level=logging.INFO)

GCP_PROJECT = os.getenv("GCP_PROJECT")
GCP_REGION = os.getenv("GCP_REGION")
GOOGLE_MAP_KEY = os.getenv("GOOGLE_MAP_KEY")

vertexai.init(project=GCP_PROJECT, location=GCP_REGION)

### Agents

In [2]:
from google.adk.agents import Agent, LlmAgent, SequentialAgent
from google.adk.tools import google_search
from vertexai.preview import reasoning_engines
from IPython.display import Markdown, display

search_agent = LlmAgent(
    name="search_agent",
    model="gemini-2.0-flash",
    instruction="Use the Google Search tool to find information about a topic.",
    tools=[google_search],
    output_key="search_results",
)

critique_agent = LlmAgent(
    name="critique_agent",
    model="gemini-2.0-flash",
    instruction="Critique the search results stored in the state key 'search_results'.",
    output_key="critique_results",
)

refine_agent = LlmAgent(
    name="refine_agent",
    model="gemini-2.5-flash",
    instruction="Refine the search results based on the critique stored in the state key 'critique_results'. This will be used as final output to the end user.",
    output_key="refined_results",
)

greeter_agent = Agent(
    name="greeter_agent",
    model="gemini-2.0-flash",
    instruction="Greet the user and ask for a topic to search, then pass to 'search_agent'.",
)

search_sequence = SequentialAgent(
    name="search_sequence",
    sub_agents=[greeter_agent, search_agent, critique_agent, refine_agent],
)


In [3]:
app = reasoning_engines.AdkApp(
    agent=search_sequence,
    enable_tracing=False,
)

user_id = "test-user-id"
session = app.create_session(user_id=user_id)

for event in app.stream_query(
    user_id=user_id,
    session_id = session.id,
    message="What does it mean to be human?",
):
  lastevent = event

display(Markdown(lastevent["content"]["parts"][0]["text"]))

Being human is a profoundly complex and multifaceted concept, encompassing biological, philosophical, social, and emotional dimensions. Here's an expanded overview, incorporating various perspectives to provide a more nuanced understanding:

While specific citations from the original search were not provided, a comprehensive answer would typically draw from sources such as biological anthropology texts, philosophical works (e.g., Aristotle, Descartes, existentialist philosophers), sociological studies, psychology research, and cultural anthropology.

### 1. Biological Perspective: Our Animal Nature

*   **Classification:** Humans are scientifically classified as *Homo sapiens*, a unique species of culture-bearing primates within the genus *Homo*.
*   **Distinctive Traits:** We are distinguished by a highly developed brain, enabling abstract reasoning and articulate speech. Our unique upright posture frees our hands for tool use and complex manipulation of our environment.
*   **Fundamental Processes:** From a purely biological standpoint, human life involves self-sustaining processes like homeostasis, organization, metabolism, growth, adaptation, response to stimuli, and reproduction.

### 2. Philosophical Perspective: The Quest for Meaning and Self

*   **Rationality and Consciousness:** A defining characteristic is the capacity for abstract rational thought, which underpins language, culture, and complex ideologies. Philosophers like Aristotle emphasized humans as "rational animals" (*animal rationale*). René Descartes famously posited "I think, therefore I am," highlighting consciousness and self-awareness as central to human existence.
*   **Existentialism and Meaning:** Beyond inherent rationality, being human involves grappling with existential questions of purpose, meaning, and our place in the universe. Philosophers like Jean-Paul Sartre and Albert Camus explored the freedom and burden of creating our own meaning in an often-absurd world.
*   **Humanism:** Many philosophical traditions, particularly humanism, place human values, dignity, and capacity for self-realization at the center, emphasizing our potential for good.
*   **Moral and Ethical Capacity:** Uniquely, humans possess the ability to differentiate between right and wrong, to establish ethical frameworks, and to act based on moral reasoning, often debating the source and nature of these moral principles.

### 3. Social Perspective: Connection, Culture, and Community

*   **Inherent Sociality:** Humans are fundamentally social beings, driven to form complex communities, from families and tribes to nations and global networks. As the expression "no man is an island" suggests, our identities and lives are deeply intertwined with others.
*   **Culture-Bearing and Shaping:** We are both products and creators of culture. Culture encompasses shared beliefs, values, customs, behaviors, and artifacts that shape our understanding of the world and our interactions within it. This capacity for cumulative cultural learning is unparalleled.
*   **Complex Social Structures:** Human societies are characterized by intricate divisions of labor, legal systems, and institutions (e.g., educational, political, economic) that organize and regulate collective life.
*   **Impact on the World:** Our collective actions have a profound and often transformative impact on the natural environment and global systems.

### 4. Emotional Perspective: The Spectrum of Human Experience

*   **Rich Emotional Landscape:** To be human is to experience a vast spectrum of emotions – joy, sorrow, love, grief, anger, hope, fear, regret, and myriad subtle variations. These emotions are not merely reactions but are integral to our cognitive processes, decision-making, and connection with others.
*   **Empathy and Compassion:** Empathy, the ability to understand and share the feelings of others, is a cornerstone of human social bonding and is crucial for fostering compassion, altruism, and supportive communities.
*   **Emotional Awareness:** The capacity for self-reflection extends to our emotions, allowing us to understand our inner states and develop emotional intelligence, which contributes to personal growth and fulfillment.

### 5. Key Human Qualities & Paradoxes:

*   **Consciousness:** The remarkable capacity to be aware of oneself, one's thoughts, feelings, and surroundings.
*   **Creativity:** The unique power to imagine, innovate, and express novel ideas, from art and music to scientific discoveries and technological advancements.
*   **Resilience:** The innate ability to adapt, recover, and even grow in the face of adversity, trauma, and challenges.
*   **The "Dark Side":** While humans possess immense capacity for good, love, and cooperation, we also demonstrate a capacity for cruelty, selfishness, irrationality, and destructive behavior. A complete understanding of human nature acknowledges this duality – our potential for both great good and great harm.
*   **Search for Purpose:** An enduring drive to find significance, meaning, and contribution in one's life.

### 6. The Evolving Definition:

*   **Cultural Relativity:** What it means to be human can vary across different cultures and historical periods, influenced by prevailing beliefs, values, and social structures. There isn't a single, universal definition that applies identically everywhere.
*   **Future Trajectories:** The concept of "human" itself is continually evolving, particularly with advancements in areas like genetic engineering, artificial intelligence, and transhumanism, which raise new questions about our biological and cognitive limits, and the potential future forms of intelligence and existence.

In essence, being human is a continuous, dynamic journey shaped by our unique biological heritage, our profound capacity for thought and feeling, our inescapable social bonds, and our ongoing pursuit of meaning and purpose in a complex world. It is a paradox of rationality and irrationality, creation and destruction, profound empathy and profound cruelty.